HE block


In [2]:
!pip install mat4py
from google.colab import drive
import os
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [0]:
import numpy as np
from scipy.integrate import odeint, quad
from scipy.misc import derivative
import math

class ECM_HE:
    def __init__(self, P, HE_Cap=12240, HE_R0=0.0248, HE_R1=0.0081, HE_R2=0.0282, HE_C1 = 224.4569, HE_C2 = 1929.2 ,HE_Temp_cell = 300,\
        HE_SOC = 0.9, HE_V1 = 0, HE_V2 = 0, HE_Tamb = 300, Time_diff = 1, HE_OCV = 4.10, HE_Vt = 4.10, HE_Xk = [[0.9], [0], [0]]) : #V10, V20 initial value of V1, V2
        self.I = []
        # self.I = i
        self.P = []
        self.P = P
        #print(self.I)
        self.T = []
        if isinstance(HE_Temp_cell, int):
          self.T.append(HE_Temp_cell) 
        else:
          self.T = HE_Temp_cell
        self.Tamb = HE_Tamb
        self.Time_diff = Time_diff
        # ###########parameter for 2 RC model #######################
        #initial value
        #self.SOC_ini = soc_ini
        #self.V10 = v10
        #self.V20 = v20
        #predifined parameter
        #self.Cap = []
        self.Cap = HE_Cap#12240 #17280                                              # unit:As

        #ECM elements parameter
        self.R0 = HE_R0#0.0248
        self.R1 = HE_R1#0.0081
        self.R2 = HE_R2#0.0282#3.6291e-2
        self.C1 = HE_C1
        self.C2 = HE_C2
        
        self.V1 = []                              #V1
        if isinstance(HE_V1,list) == False:                   
          self.V1.append([0, HE_V1])
        else:
          self.V1 = HE_V1
        
        self.V2 = []                              #V2 
        if isinstance(HE_V2,list) == False:                   
          self.V2.append([0, HE_V2])
        else:
          self.V2 = HE_V2
        
        self.OCV = []                                   #OCV
        if isinstance(HE_OCV,list) == False: 
          #print('#')
          self.OCV.append([0, HE_OCV])
        else:
          #print('####')
          self.OCV = HE_OCV

        
        self.SOC = []                              #SOC
        if isinstance(HE_SOC, list) == False:                   
          self.SOC.append([0, HE_SOC])
        else:
          self.SOC = HE_SOC
        
        self.Vt = []                               #Vt
        if isinstance(HE_Vt,list) == False:                   
          self.Vt.append([[0, HE_Vt]])
        else:
          self.Vt = HE_Vt
        #elf.OCV.append([0, 4.10])                                                  # initial value for OCV
        self.etha = 0.99         
       
        #self.SOC.append([0,self.SOC_ini])
        
       
        self.Xk = []                                 #Xk
        if isinstance(HE_SOC,list) == False:                   
          self.Xk.append([0, HE_Xk])
          #print('##',self.Xk)
        else:
          #print('############')
          self.Xk = HE_Xk
        # self.Xk = np.array(self.Xk)
        #OCV-SOC nonlinearity parameter
        self.K0 = 2.97#3.641
        self.K1 = 3.662#0.3696
        self.K2 = -8.268#-0.6644
        self.K3 = 10.13#0.6489
        self.K4 = -4.348#1.486e-5
        #self.K5 = #0.06265

        ###############parameter for thermal model###########
        self.V_avg = []
        self.h = 20*0.092#0.004464*72*57
        self.C_thermal_cell = 0.85*67.6

    def ocv_upd(self, num):
        #OCV_cur = self.K0 + self.K1 * self.SOC[num][1] + self.K2 * np.power(self.SOC[num][1], 2) + self.K3 * np.power(self.SOC[num][1], 3) + self.K4 / self.SOC[num][1] + self.K5 * np.log(self.SOC[num][1])
        OCV_cur = self.K0 + self.K1 * self.SOC[num][1] + self.K2 * np.power(self.SOC[num][1], 2) + self.K3 * np.power(self.SOC[num][1], 3)+ self.K4 * np.power(self.SOC[num][1], 4)
        return OCV_cur

    def r0_upd(self):
        x=self.T[-1]-273
        y=self.SOC[-1][1]
        p00 =     0.05454  
        p10 =   -0.001999
        p01 =  -0.0001144
        p20 =   3.211e-05
        p11 =  -3.607e-07  
        p02 =   9.071e-07 
        self.R0 = p00 + p10*x + p01*y + p20*(x**2) + p11*x*y + p02*(y**2)
        return self.R0
        



    def c1_upd(self):
        x=self.T[-1]-273
        y=self.SOC[-1][1]
        p1 =       -6105  
        p2 =   1.527e+04  
        p3 =  -1.348e+04  
        p4 =        4680  
        p5 =      -190.4 
        self.C1 = p1*(y**4) + p2*(y**3) + p3*(y**2) + p4*y + p5
        self.C1 = 6.9*(x-25) + self.C1 
        #self.C1 =224.4569
        return self.C1

    def r1_upd(self):
        x=self.T[-1]-273
        y=self.SOC[-1][1]
        #self.R1 = 0.0081
        p00 =    0.006423  
        p10 =    -0.00293 
        p01 =   0.0006537  
        p20 =    0.001389 
        p11 =  -0.0005024  
        p02 =   0.0007236  
        self.R1 = p00 + p10*x + p01*y + p20*(x**2) + p11*x*y + p02*(y**2)
        return self.R1

    def c2_upd(self):
        x=self.T[-1]-273
        y=self.SOC[-1][1]
        p1 =       -3819  
        p2 =   1.823e+04 
        p3 =  -2.367e+04 
        p4 =   1.066e+04  
        p5 =       541.9 
        self.C2 = p1*(y**4) + p2*(y**3) + p3*(y**2) + p4*y + p5
        self.C2 = 34.1*(x-25) + self.C2  
        #self.C2 =  1929.2
        return self.C2

    def r2_upd(self):
        x=self.T[-1]-273
        y=self.SOC[-1][1]
        p1 =       1.583  
        p2 =      -4.712 
        p3 =       5.039  
        p4 =      -2.354
        p5 =      0.4611 
        p6 =    0.005176 
        self.R2 = p1*(y**5) + p2*(y**4) + p3*(y**3) + p4*(y**2) + p5*y + p6
        self.R2 = -0.0008935*(x-25) + self.R2

        return self.R2

    def dV1_cal(self, V1,  a):
        dV1dt = (-1 / (self.R1 * self.C1)) * V1 + 1 / self.C1 * a
        return dV1dt

    def dV2_cal(self, V2,  a):
        dV2dt = (-1 / (self.R2 * self.C2)) * V2 + 1 / self.C2 * a
        return dV2dt

    def cur_dert(self, num):
        a = self.R0
        b = self.OCV[-1][1] + self.V1[-1][1] + self.V2[-1][1]
        #print(self.P)
        c = -self.P[-1][1]    #######delete the 2nd index 
        inter = np.sqrt((b**2)-  4*a*c)
        #if inter < 0 :
        #    cur = (-b ) /(2*a)
        #else:
        cur =-(-b + inter) / (2*a)
        #print('cur',cur)
        return cur

    def twoRCECM(self, num, current):
        self.OCV.append([num , self.ocv_upd(num)])
        #ECM discrete state vector space parameter
        self.Ad = []
        self.Ad = np.array([[1,0,0],[0, np.exp(-self.Time_diff/self.R1/self.C1), 0], [0, 0, np.exp(-self.Time_diff/self.R2/self.C2)]])
   
        self.Bd = []
        self.Bd = np.array([[(self.etha*self.Time_diff)/self.Cap], [(1-np.exp(-self.Time_diff/self.R1/self.C1))*self.R1], [(1-np.exp(-self.Time_diff/self.R2/self.C2))*self.R2]])
        self.Xk.append([num+1, np.ndarray.tolist(np.dot(self.Ad , self.Xk[num][1]) + np.dot(self.Bd , current))])
        self.SOC.append([num + 1, self.Xk[num+1][1][0][0]])
        self.V1.append([num + 1, self.Xk[num+1][1][1][0]])
        self.V2.append([num + 1, self.Xk[num+1 ][1][2][0]])
        self.Vt.append([num, self.OCV[num + 1 ][1] + self.V1[num+1][1] + self.V2[num+1][1] + self.R0*current])
        #self.thermal(num, current)
        #self.R0 = self.r0_upd()        
        #self.R1 = self.r1_upd()
        #self.R2 = self.r2_upd()
        
        #self.C1 = self.c1_upd()
        #self.C2 = self.c2_upd()
        return self.Vt, self.SOC, self.V1, self.V2, self.T, self.Cap, self.R0, self.R1, self.R2, self.C1, self.C2, self.OCV, self.Xk
    
    def aging(self,SOC_ini,SOC_end, V_ini, V_end):
        # calendrc aging is related to voltage(volts) and absolute temperature(kelvin)
        #alpha_cap = (7.543 * self.OCV[-1][1] - 23.75) * np.power(10, 6) * np.exp(-6976 / self.T[-1])
        #alpha_res = (5.270 * self.OCV[-1][1] - 16.32) * np.power(10, 5) * np.exp(-5986 / self.T[-1])
       # cyclinc aging
        Q = self.Cap *(1-self.SOC[-1][1]) # t1 is cycle time
        deltaDOD = np.abs(SOC_ini - SOC_end)
        V_arv = 0.5 * (V_ini - V_end)
        beta_cap = 7.348 * np.power(10, -3) * ((V_arv - 3.667) ** 2) + 7.600 * np.power(10, -4) + 4.081 * np.power(10,-3) * deltaDOD
        beta_res = 2.153 * np.power(10, -4) * ((V_arv - 3.725) ** 2) - 1.521 * np.power(10, -5) + 2.798 * np.power(10,-4) * deltaDOD
       # total influence, t in days, Q is ampere hour
        Cap = 1  - beta_cap * np.sqrt(Q) #- alpha_cap * np.power(t, 0.75) #t's unit is day
        Res = 1  - beta_res * np.sqrt(Q) #+ alpha_res * np.power(t, 0.75)
        return Cap, Res
      
    def thermal(self, current):
        heat = 72*57 *((self.V1[-1][1]**2)/self.R1 + (self.V2[-1][1]**2)/self.R2 + self.R0 * (current**2)) + 0.0007645 *57*72* current * self.T[-1]
        dissipation = self.h * (self.T[-1]-self.Tamb)
        t_delta = (heat-dissipation) / (72*57 *self.C_thermal_cell)
        self.T.append(self.T[-1]+t_delta)
        return self.T

HP block

In [0]:
import numpy as np
from scipy.integrate import odeint, quad
from scipy.misc import derivative
#import math

class ECM_HP:
   
    def __init__(self, P, HP_Cap=10440, HP_R0=4.5588e-3, HP_R1=0.0013, HP_R2=0.0040, HP_C1 = 2249.5, HP_C2 = 18869 ,HP_Temp_cell = 300, HP_SOC = 0.5, HP_V1 = 0, HP_V2 = 0, HP_Tamb = 300, Time_diff = 1, HP_OCV = 2.7, HP_Vt = 2.7, HP_Xk = [[0.5], [0], [0]]) : #V10, V20 initial value of V1, V2
        self.I = []
        # self.I = i
        self.P = []
        self.P = P
        #print(self.I)
        self.T = []
        if isinstance(HP_Temp_cell, int):
          self.T.append(HP_Temp_cell) 
        else:
          self.T = HP_Temp_cell
        self.Tamb = HP_Tamb
        self.Time_diff = Time_diff
        # ###########parameter for 2 RC model #######################
        #initial value
        #self.SOC_ini = soc_ini
        #self.V10 = v10
        #self.V20 = v20
        #predifined parameter
        #self.Cap = []
        self.Cap = HP_Cap#12240 #17280                                              # unit:As

        #ECM elements parameter
        self.R0 = HP_R0#0.0248
        self.R1 = HP_R1#0.0081
        self.R2 = HP_R2#0.0282#3.6291e-2
        self.C1 = HP_C1
        self.C2 = HP_C2
        
        self.V1 = []                              #V1
        if isinstance(HP_V1,list) == False:                   
          self.V1.append([0, HP_V1])
        else:
          self.V1 = HP_V1
        
        self.V2 = []                              #V2 
        if isinstance(HP_V2,list) == False:                   
          self.V2.append([0, HP_V2])
        else:
          self.V2 = HP_V2
        
        self.OCV = []                                   #OCV
        if isinstance(HP_OCV,list) == False: 
          #print('#')
          self.OCV.append([0, HP_OCV])
        else:
          #print('####')
          self.OCV = HP_OCV

        
        self.SOC = []                              #SOC
        if isinstance(HP_SOC, list) == False:                   
          self.SOC.append([0, HP_SOC])
        else:
          self.SOC = HP_SOC
        
        self.Vt = []                               #Vt
        if isinstance(HP_Vt,list) == False:                   
          self.Vt.append([[0, HP_Vt]])
        else:
          self.Vt = HP_Vt
        #elf.OCV.append([0, 4.10])                                                  # initial value for OCV
        self.etha = 0.99         
       
        #self.SOC.append([0,self.SOC_ini])
        
       
        self.Xk = []                                 #Xk
        if isinstance(HP_SOC,list) == False:                   
          self.Xk.append([0, HP_Xk])
          #print('##',self.Xk)
        else:
          #print('############')
          self.Xk = HP_Xk
        # self.Xk = np.array(self.Xk)
        #OCV-SOC nonlinearity parameter
        self.K0 = 2.306
        self.K1 = 0.003702
        self.K2 = 5.382e-5
        self.K3 = -1.794e-6
        self.K4 = 1.309e-8

        ###############parameter for thermal model###########
        self.V_avg = []
        self.h = 20*0.092#0.004464*72*57
        self.C_thermal_cell = 0.85*150.8

    def ocv_upd(self, num):
        #OCV_cur = self.K0 + self.K1 * self.SOC[num][1] + self.K2 * np.power(self.SOC[num][1], 2) + self.K3 * np.power(self.SOC[num][1], 3) + self.K4 / self.SOC[num][1] + self.K5 * np.log(self.SOC[num][1])
        OCV_cur = self.K0 + self.K1 * self.SOC[num][1] + self.K2 * np.power(self.SOC[num][1], 2) + self.K3 * np.power(self.SOC[num][1], 3)+ self.K4 * np.power(self.SOC[num][1], 4)
        return OCV_cur

    def r0_upd(self):
        x=self.T[-1]-273
        y=self.SOC[-1][1]
        p1 =  -2.783e-09 
        p2 =   4.993e-07  
        p3 =  -3.129e-05  
        p4 =    0.001997 
        self.R0 = p1*(y**3) + p2*(y**2) + p3*y + p4
        self.R0 =  -4.387e-05 *(x-25) + self.R0 
        return self.R0


    def c1_upd(self):
        x=self.T[-1]-273
        y=self.SOC[-1][1]
        p1 =  -1.662e+04  
        p2 =    2.68e+04 
        p3 =  -1.324e+04  
        p4 =        4415 
        self.C1 = p1*(y**3) + p2*(y**2) + p3*y + p4
        self.C1 = 46.92 *(x-25) + self.C1 
        #self.C1 =224.4569
        return self.C1

    def r1_upd(self):
        x=self.T[-1]-273
        y=self.SOC[-1][1]
        #self.R1 = 0.0081
        p1 =    0.007192  
        p2 =    -0.01001  
        p3 =    0.003639  
        p4 =    0.001028  
        self.R1 = p1*(y**3) + p2*(y**2) + p3*y + p4
        self.R1 = -4.416e-05 *(x-25) + self.R1 
        return self.R1

    def c2_upd(self):
        x=self.T[-1]-273
        y=self.SOC[-1][1]        
        p1 =   -1.11e+05  
        p2 =   1.061e+05  
        p3 =       -1808  
        p4 =   1.274e+04  
        self.C2 = p1*(y**3) + p2*(y**2) + p3*y + p4
        self.C2 = 782.5*(x-25) + self.C2  
        #self.C2 =  1929.2
        return self.C2

    def r2_upd(self):
        x=self.T[-1]-273
        y=self.SOC[-1][1]
        p1 =     0.01357 
        p2 =    -0.01614 
        p3 =    0.007044   
        self.R2 = p1*(y**2) + p2*y + p3
        self.R2 = --9.067e-05*(x-25) + self.R2

        return self.R2

    def dV1_cal(self, V1,  a):
        dV1dt = (-1 / (self.R1 * self.C1)) * V1 + 1 / self.C1 * a
        return dV1dt

    def dV2_cal(self, V2,  a):
        dV2dt = (-1 / (self.R2 * self.C2)) * V2 + 1 / self.C2 * a
        return dV2dt

    def cur_dert(self, num):
        a = self.R0
        b = self.OCV[-1][1] + self.V1[-1][1] + self.V2[-1][1]
        c = -self.P[-1][1]    #######delete the 2nd index 
        inter = np.sqrt((b**2)-  4*a*c)
        if inter < 0 :
            cur = (-b ) /(2*a)
        else:
            cur =-(-b + inter) / (2*a)
        #print('cur',cur)
        return cur

    def twoRCECM(self, num, current):
        self.OCV.append([num , self.ocv_upd(num)])
        #ECM discrete state vector space parameter
        self.Ad = []
        self.Ad = np.array([[1,0,0],[0, np.exp(-self.Time_diff/self.R1/self.C1), 0], [0, 0, np.exp(-self.Time_diff/self.R2/self.C2)]])
   
        self.Bd = []
        self.Bd = np.array([[(self.etha*self.Time_diff)/self.Cap], [(1-np.exp(-self.Time_diff/self.R1/self.C1))*self.R1], [(1-np.exp(-self.Time_diff/self.R2/self.C2))*self.R2]])
        self.Xk.append([num+1, np.ndarray.tolist(np.dot(self.Ad , self.Xk[num][1]) + np.dot(self.Bd , current))])
        self.SOC.append([num + 1, self.Xk[num+1][1][0][0]])
        self.V1.append([num + 1, self.Xk[num+1][1][1][0]])
        self.V2.append([num + 1, self.Xk[num+1 ][1][2][0]])
        self.Vt.append([num, self.OCV[num + 1 ][1] + self.V1[num+1][1] + self.V2[num+1][1] + self.R0*current])
        #self.thermal(num, current)
        #self.R0 = self.r0_upd()        
        #self.R1 = self.r1_upd()
        #self.R2 = self.r2_upd()
        
        #self.C1 = self.c1_upd()
        #self.C2 = self.c2_upd()
        return self.Vt, self.SOC, self.V1, self.V2, self.T, self.Cap, self.R0, self.R1, self.R2, self.C1, self.C2, self.OCV, self.Xk
    
    def aging(self,SOC_ini,SOC_end, V_ini, V_end):
        # calendrc aging is related to voltage(volts) and absolute temperature(kelvin)
        #alpha_cap = (7.543 * self.OCV[-1][1] - 23.75) * np.power(10, 6) * np.exp(-6976 / self.T[-1])
        #alpha_res = (5.270 * self.OCV[-1][1] - 16.32) * np.power(10, 5) * np.exp(-5986 / self.T[-1])
       # cyclinc aging
        Q = self.Cap *(1-self.SOC[-1][1]) # t1 is cycle time
        deltaDOD = np.abs(SOC_ini - SOC_end)
        V_arv = 0.5 * (V_ini - V_end)
        beta_cap = 7.348 * np.power(10, -3) * ((V_arv - 3.667) ** 2) + 7.600 * np.power(10, -4) + 4.081 * np.power(10,-3) * deltaDOD
        beta_res = 2.153 * np.power(10, -4) * ((V_arv - 3.725) ** 2) - 1.521 * np.power(10, -5) + 2.798 * np.power(10,-4) * deltaDOD
       # total influence, t in days, Q is ampere hour
        Cap = 1  - beta_cap * np.sqrt(Q) #- alpha_cap * np.power(t, 0.75) #t's unit is day
        Res = 1  - beta_res * np.sqrt(Q) #+ alpha_res * np.power(t, 0.75)
        return Cap, Res
      
    def thermal(self, current):
        heat = 72*57 *((self.V1[-1][1]**2)/self.R1 + (self.V2[-1][1]**2)/self.R2 + self.R0 * (current**2)) + 0.0007645 *57*72* current * self.T[-1]
        dissipation = self.h * (self.T[-1]-self.Tamb)
        t_delta = (heat-dissipation) / (72*57 *self.C_thermal_cell)
        self.T.append(self.T[-1]+t_delta)
        return self.T
   

Main bat block

In [0]:
import numpy as np
import tensorflow as tf
np.random.seed(1)
tf.set_random_seed(1)
# Deep Q Network off-policy
class DeepQNetwork:
    def __init__(
            self,
            n_actions,
            n_features,
            learning_rate=0.00025,
            reward_decay=0.99,
            e_greedy=1,
            replace_target_iter=200,
            memory_size=1000,
            batch_size=32,
            e_greedy_increment=-0.01,
            output_graph=False,
    ):
        self.n_actions = n_actions
        self.n_features = n_features
        self.lr = learning_rate
        self.gamma = reward_decay
        self.epsilon_max = e_greedy
        self.replace_target_iter = replace_target_iter
        self.memory_size = memory_size
        self.batch_size = batch_size
        self.epsilon_increment = e_greedy_increment
        self.epsilon = self.epsilon_max #if e_greedy_increment is not None else self.epsilon_max

        # total learning step
        self.learn_step_counter = 0

        # initialize zero memory [s, a, r, s_]
        self.memory = np.zeros((self.memory_size, n_features * 2 + 2))

        # consist of [target_net, evaluate_net]
        self._build_net()
		
        t_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='target_net')
        e_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='eval_net')

        with tf.variable_scope('hard_replacement',reuse = tf.AUTO_REUSE):
            self.target_replace_op = [tf.assign(t, e) for t, e in zip(t_params, e_params)]

        self.sess = tf.Session()

        if output_graph:
            # $ tensorboard --logdir=logs
            tf.summary.FileWriter("logs/", self.sess.graph)

        self.sess.run(tf.global_variables_initializer())
        #self.cost_his = []
        # total cost and reward:
        self.cost_total = 0
        

    def _build_net(self):
        # ------------------ all inputs ------------------------
        self.s = tf.placeholder(tf.float32, [None, self.n_features], name='s')  # input State
        self.s_ = tf.placeholder(tf.float32, [None, self.n_features], name='s_')  # input Next State
        self.r = tf.placeholder(tf.float32, [None, ], name='r')  # input Reward
        self.a = tf.placeholder(tf.int32, [None, ], name='a')  # input Action

        w_initializer, b_initializer = tf.random_normal_initializer(0., 0.3), tf.constant_initializer(0.1)

        # ------------------ build evaluate_net ------------------
        with tf.variable_scope('eval_net', reuse = tf.AUTO_REUSE):
            e1 = tf.layers.dense(inputs = self.s, units =150, activation = tf.nn.relu, kernel_initializer=w_initializer,
                                 bias_initializer=b_initializer, name='e1')
            e2 = tf.layers.dense(inputs = e1, units = 100, activation = tf.nn.relu, kernel_initializer=w_initializer,
                                          bias_initializer=b_initializer, name='e2')
            e3 = tf.layers.dense(inputs = e2, units = 50, activation = tf.nn.relu, kernel_initializer=w_initializer,
                                          bias_initializer=b_initializer, name='e3')
            self.q_eval = tf.layers.dense(inputs = e3, units = self.n_actions, kernel_initializer=w_initializer,
                                          bias_initializer=b_initializer, name='q')

        # ------------------ build target_net ------------------
        with tf.variable_scope('target_net',reuse = tf.AUTO_REUSE):
            t1 = tf.layers.dense(inputs = self.s_, units = 150, activation = tf.nn.relu, kernel_initializer=w_initializer,
                                 bias_initializer=b_initializer, name='t1')
            t2 = tf.layers.dense(inputs = t1, units = 100, activation = tf.nn.relu, kernel_initializer=w_initializer,
                                 bias_initializer=b_initializer, name='t2')
            t3 = tf.layers.dense(inputs = t2, units = 50, activation = tf.nn.relu, kernel_initializer=w_initializer,
                                          bias_initializer=b_initializer, name='t3')
            self.q_next = tf.layers.dense(inputs = t3, units = self.n_actions, kernel_initializer=w_initializer,
                                          bias_initializer=b_initializer, name='q2')

        with tf.variable_scope('q_target',reuse = tf.AUTO_REUSE):
            q_target = self.r + self.gamma * tf.reduce_max(self.q_next, axis=1, name='Qmax_s_')    # shape=(None, )
            self.q_target = tf.stop_gradient(q_target)
        with tf.variable_scope('q_eval',reuse = tf.AUTO_REUSE):
            a_indices = tf.stack([tf.range(tf.shape(self.a)[0], dtype=tf.int32), self.a], axis=1)
            self.q_eval_wrt_a = tf.gather_nd(params=self.q_eval, indices=a_indices)    # shape=(None, )
        with tf.variable_scope('loss',reuse = tf.AUTO_REUSE):
            self.loss = tf.reduce_mean(tf.squared_difference(self.q_target, self.q_eval_wrt_a, name='TD_error'))
        with tf.variable_scope('train',reuse = tf.AUTO_REUSE):
            self._train_op = tf.train.RMSPropOptimizer(self.lr).minimize(self.loss)

    def store_transition(self, s, a, r, s_):
        if not hasattr(self, 'memory_counter'):
            self.memory_counter = 0
        transition = np.hstack((s, [a, r], s_))
        # replace the old memory with new memory
        #print(transition)
        index = self.memory_counter % self.memory_size
        self.memory[index, :] = transition
        self.memory_counter += 1

    def choose_action(self, observation):
        # to have batch dimension when feed into tf placeholder

        observation = observation[np.newaxis, :]
        #observation = observation
        if np.random.uniform() < self.epsilon:
            #print("1st loop")
            
            action = np.random.randint(0, self.n_actions)
        else:
            #print("2nd loop")
            # forward feed the observation and get q value for every actions
            actions_value = self.sess.run(self.q_eval, feed_dict={self.s: observation})
            action = np.argmax(actions_value)
            
        return action

    def learn(self):
        # check to replace target parameters
        if self.learn_step_counter % self.replace_target_iter == 0:
            self.sess.run(self.target_replace_op)
            print('\ntarget_params_replaced\n')

        # sample batch memory from all memory
        if self.memory_counter > self.memory_size:
            sample_index = np.random.choice(self.memory_size, size=self.batch_size)
        else:
            sample_index = np.random.choice(self.memory_counter, size=self.batch_size)
        batch_memory = self.memory[sample_index, :]

        _, cost = self.sess.run(
            [self._train_op, self.loss],
            feed_dict={
                self.s: batch_memory[:, :self.n_features],
                self.a: batch_memory[:, self.n_features],
                self.r: batch_memory[:, self.n_features + 1],
                self.s_: batch_memory[:, -self.n_features:],
            })

        #self.cost_his.append(cost)
		
        # increasing epsilon
        self.epsilon = self.epsilon + self.epsilon_increment if 0.2 < self.epsilon < self.epsilon_max else 0.2
        self.learn_step_counter += 1
        return cost
#    def plot_cost(self):
#        import matplotlib.pyplot as plt
#        plt.plot(np.arange(len(self.cost_his)), self.cost_his)
#       plt.ylabel('Cost')
#        plt.xlabel('training steps')
#        plt.show()
#        plt.savefig('/rwthfs/rz/cluster/home/vv465559/cui/cost.png')
	
    #def cost(self):
        #for i in range(len(self.cost_his)):
            #self.cost_total = self.cost_total + self.cost_his[i]
        #return self.cost_total
	
    def save(self):
        saver = tf.train.Saver()
        save_path = saver.save(self.sess, '/content/gdrive/My Drive/Colab Notebooks/trained_model.ckpt')

In [0]:
#from ECM import Battery_ECM as ECM_HE
#from ECM_HP import Battery_ECM as ECM_HP
import random as rd
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd

       

class Env_battery:

    def __init__(self, Power_total, HE_power_vector, HP_power_vector, HEcur, HPcur, Speed_vector ):  ####### self.Current_total is log data or reults from EV model
        self.action_space = ['0', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', '1.0']   #delete the comma in the end 
        self.n_actions = len(self.action_space)
        self.n_states = 6

        self.Power_total = Power_total
        self.HE_power_vector = HE_power_vector
        self.HP_power_vector = HP_power_vector
        self.HE_battery = ECM_HE([0,0])            #using seperate HE and HP battery model
        #print("initialHE", self.HE_current_vector)
        self.HP_battery = ECM_HP([0,0])
        #print("initialHP", self.HP_current_vector)
        self.HE_cell_series =72 #92      #52.2KWH HE and 3.5KWH HP
        self.HE_cell_parallel = 57 #14
        self.HE_cell_num = self.HE_cell_series * self.HE_cell_parallel 
        self.HP_cell_series = 146 #122
        self.HP_cell_parallel = 3 #17
        self.HP_cell_num = self.HP_cell_series * self.HP_cell_parallel 
        self.Speed_vector = Speed_vector
        self.HE_P_max_dis = 200000                 #power limit
        self.HE_P_max_cha = -100000
        self.HP_P_max_dis = 200000
        self.HP_P_max_cha = -200000
        self.HE_current = HEcur
        self.HP_current = HPcur

		
    def reset(self):
        self.HE_power_vector = []
        self.HP_power_vector = []
        self.HE_battery = ECM_HE(self.HE_power_vector)
        self.HP_battery = ECM_HP(self.HP_power_vector)
        return (np.array([self.HE_battery.SOC[0][1], self.HP_battery.SOC[0][1],0,0,0,0]))  # return the reset initial state
    
    def dcdc(self,x):
        p1 =     0.01556
        p2 =      -1.444
        p3 =       388.9 
        y = p1*(x**2) + p2*x + p3
        return y
    
    def step(self, action, num, HE_Cap, HE_R0, HE_R1, HE_R2, HE_C1, HE_C2, HE_Temp_cell, HE_SOC, HE_V1, HE_V2, HE_OCV, HE_Vt, HE_Xk \
               ,HP_Cap, HP_R0, HP_R1, HP_R2, HP_C1, HP_C2, HP_Temp_cell, HP_SOC, HP_V1, HP_V2, HP_OCV, HP_Vt, HP_Xk, aging_flag, aging_init_time):
        ######power distribution
        self.HE_power = self.Power_total[num][1] * action * 0.1
        self.HP_power = self.Power_total[num][1] - self.HE_power
        HP_pow = self.HP_power / self.HP_cell_num
        HE_pow = self.HE_power / self.HE_cell_num
        self.HE_power_vector.append([num, HE_pow])
        self.HP_power_vector.append([num, HP_pow])
        #Battery_ECM(power_vector, HE_R0=R0, HE_R1=R1, HE_R2=R2, HE_C1 = C1, HE_C2= C2,Temp_cell = T, SOC = SOC, V1 = V1, V2 = V2,  OCV = OCV, Vt = Vt, Xk = Xk)
        self.HE_battery = ECM_HE(self.HE_power_vector, HE_Cap=HE_Cap, HE_R0=HE_R0, HE_R1=HE_R1, HE_R2=HE_R2, HE_C1 = HE_C1, HE_C2= HE_C2,HE_Temp_cell = HE_Temp_cell, HE_SOC = HE_SOC, HE_V1 = HE_V1, HE_V2 = HE_V2,  HE_OCV = HE_OCV, HE_Vt = HE_Vt, HE_Xk = HE_Xk)
        self.HP_battery = ECM_HP(self.HP_power_vector, HP_Cap=HP_Cap, HP_R0=HP_R0, HP_R1=HP_R1, HP_R2=HP_R2, HP_C1 = HP_C1, HP_C2= HP_C2,HP_Temp_cell = HP_Temp_cell, HP_SOC = HP_SOC, HP_V1 = HP_V1, HP_V2 = HP_V2,  HP_OCV = HP_OCV, HP_Vt = HP_Vt, HP_Xk = HP_Xk)
        HE_cur = self.HE_battery.cur_dert(num)
        HP_cur = self.HP_battery.cur_dert(num)
        self.HE_current.append([num, HE_cur])
        self.HP_current.append([num, HP_cur])
        ####DC_DC_converter_loss################################### 
        HP_cur_total = HP_cur * self.HP_cell_parallel
        dcdc_loss = self.dcdc(HP_cur_total)

        
          #self.Vt, self.SOC, self.V1, self.V2, self.T, self.Cap, self.R0, self.R1, self.R2, self.C1, self.C2, self.OCV, self.Xk
        HE_Vt, HE_SOC, HE_V1, HE_V2, HE_temp, HE_Cap, HE_R0, HE_R1, HE_R2, HE_C1, HE_C2, HE_OCV, HE_Xk = self.HE_battery.twoRCECM(num, HE_cur)            
        HP_Vt, HP_SOC, HP_V1, HP_V2, HP_temp, HP_Cap, HP_R0, HP_R1, HP_R2, HP_C1, HP_C2, HP_OCV, HP_Xk = self.HP_battery.twoRCECM(num, HP_cur)
        
        if aging_flag == 1:                         #aging updating parameter 
            HE_Cap_fac, HE_res_fac = self.HE_battery.aging(HE_SOC[aging_init_time][1], HE_SOC[-1][1], HE_OCV[aging_init_time][1], HE_OCV[-1][1])
            HP_Cap_fac, HP_res_fac = self.HP_battery.aging(HP_SOC[aging_init_time][1], HP_SOC[-1][1], HP_OCV[aging_init_time][1], HP_OCV[-1][1])
            HE_Cap = HE_Cap * HE_cap_fac
            HE_R0 = HE_R0 * HE_res_fac
            HE_R1 = HE_R1 * HE_res_fac
            HE_R2 = HE_R2 * HE_res_fac
            HP_Cap = HP_Cap * HP_cap_fac
            HP_R0 = HP_R0 * HP_res_fac
            HP_R1 = HP_R1 * HP_res_fac
            HP_R2 = HP_R2 * HP_res_fac
        Speed_norm = self.Speed_vector[num]/300     #nomalize of speed and power 
        Power_norm = self.Power_total[num][1]/45000
        next_state = np.array([HE_SOC[-1][1], HP_SOC[-1][1], Speed_norm, Power_norm, self.HE_power, self.HP_power])
        print('#######action',action)
        print('####HE_temperature####',HE_temp)
        print('####HP_temperature####',HP_temp)
        print(next_state)
	
		############################reward function###################################
        joule_res_HE = (HE_cur**2 *HE_R0 + HE_V1[-1][1]**2/HE_R1 + HE_V2[-1][1]**2/HE_R2)*self.HE_cell_num  
        joule_res_HP =(HP_cur**2 *HP_R0+ HP_V1[-1][1]**2/HP_R1 + HP_V2[-1][1]**2/HP_R2)*self.HP_cell_num
        alpha = action/10
        beta = 1 - alpha
        if self.HP_power > self.HP_P_max_dis or self.HP_power < self.HP_P_max_cha : 
            print('reward1') 
            reward_function = -abs(self.HP_power)/1000 #- joule_res
        elif self.HE_power > self.HE_P_max_dis or self.HE_power < self.HE_P_max_cha: 
            print('reward2') 
            reward_function = -abs(self.HE_power)/1000 #- joule_res
        else: 
             # HE battery soc = 0 or HP battery soc = 0
            if HE_cur < -9.6 or  HE_cur > 24 or HP_cur < -200 or  HP_cur > 200 or HE_Vt[-1][1]< 2.5 or HE_Vt[-1][1] > 4.2 or HP_Vt[-1][1]<1.5 or HP_Vt[-1][1]>2.9:
                print('reward3') 
                reward_function = -4000 - (joule_res_HE+joule_res_HP)#- joule_res
            else:
                #if (next_state[0] <= 0.1 or next_state[0] >= 0.9) or (next_state[1] <= 0.2 or next_state[1] >= 0.8 ) :
                    #print('reward4') 
                    #reward_function = -2000
                if next_state[0] <= 0.1:   
                    print('reward4') 
                    reward_function = -(0.1 - next_state[0])*100000 #- joule_res 
                elif next_state[0] >= 0.9 : 
                    print('reward5') 
                    reward_function = -(next_state[0]-0.9)*100000 #- joule_res 
                elif next_state[1] <= 0.2 : 
                    print('reward6') 
                    reward_function = -(0.2 - next_state[1])*100000 #- joule_res 
                elif next_state[1] >= 0.8 :
                    print('reward7') 
                    reward_function = -(next_state[1] - 0.8)*100000 #- joule_res 
                else:
                    #if self.Power_total[num][1] < 0:
                        #if action <3:
                            #print('reward8') 
                            #reward_function = action * 500 - joule_res
                        #else:
                            #print('reward9') 
                            #reward_function = -action * 200 - joule_res
                    #else:
                        #print('reward10') 
                        #reward_function = - (joule_res) -abs(next_state[1]-0.5)*100
                     
                    #if self.Power_total[num][1] > 0: ##discharging 
                                                 
                    print('reward8')
                    #print ('DCDC',dcdc_loss )
                    reward_function = - ( joule_res_HE+joule_res_HP+dcdc_loss)
                    #else:  ###charging
                        #print('reward6')					
                        #reward_function = - (alpha*joule_res_HE + beta*joule_res_HP) + (beta * 100) 
            
        #print([HE_cur,HP_cur,joule_res_HE,joule_res_HP,self.HE_power,self.HP_power,HE_terminal_volt[-1][1],HP_terminal_volt[-1][1]])
          #HE_R0=R0, HE_R1=R1, HE_R2=R2, HE_C1 = C1, HE_C2= C2,Temp_cell = T, SOC = SOC, V1 = V1, V2 = V2,  OCV = OCV, Vt = Vt, Xk = Xk)
        return next_state, reward_function, self.HE_power_vector, self.HP_power_vector, self.HE_current, self.HP_current \
               ,HE_Cap, HE_R0, HE_R1, HE_R2, HE_C1, HE_C2, HE_Temp_cell, HE_SOC, HE_V1, HE_V2, HE_OCV, HE_Vt, HE_Xk \
               ,HP_Cap, HP_R0, HP_R1, HP_R2, HP_C1, HP_C2, HP_Temp_cell, HP_SOC, HP_V1, HP_V2, HP_OCV, HP_Vt, HP_Xk 

DQN

DQN run

In [0]:
#from Main_ECM import Env_battery  # import environment ############
#from DQN import DeepQNetwork
import pandas as pd
from mat4py import loadmat
import numpy as np
import matplotlib.pyplot as plt
# def run_env(observation, time):
#
#     #for episode in range(2000):
#         ## initial observation
#         #observation = np.array(Env_battery.reset())
#         while True:
#             #for time in range(len(final_data) + 1) :
#                 # RL choose action based on observation
#                 action = RL.choose_action(observation)
#
#                 # RL take action and get next observation and reward
#                 observation_, reward, done = Env_battery_update.step(action, time)  #####time stamp of action
#
#                 RL.store_transition(observation, action, reward, observation_)
#
#                 RL.learn()
#
#                 # swap observation
#                 observation = observation_
#
#                 # break while loop when end of this episode
#                 if done:
#                     break
#     # end of game
#     # print('game over')
#     # env.destroy()


if __name__ == "__main__":
    # set environment

    #data = loadmat('/home/cuihan/PycharmProjects/masterthesisproject/testdata/2014-03-21_MEA_0046_80300894.mat')  # print(type(data))
    #current = data['BATT_CURRENT'][1:86378:100]
    #voltage = data['BATT_V_TOTAL'][1:86378:100]
    #speed_init = data['VEH_SPEED'][1:43189:50]
    # power = []
    data = loadmat('/content/gdrive/My Drive/Colab Notebooks/wltp.mat')
    power_init = data['P_DCL_Fahrzeug']['P_custom_wltp_1s']
    speed_init = data['P_DCL_Fahrzeug']['v_custom_wltp_1s']
    power = []
    for i in range(len(power_init)):
        #power.append([i+1, -current[i]*voltage[i]])
        power.append([i, power_init[i][0]])

    #######data_pd = pd.DataFrame(power).to_numpy()
    data_pd = power
    final_data = []
    speed = []
    cost_final= []
    #reward_total = 0
    reward_final = [] #each indicate total reward in one epoch
    for i in range(2):
        for n in range(len(data_pd)):
            final_data.append([n+i*len(data_pd), data_pd[n][1]])  # n start with 0, 17 same trips
            #speed.append(speed_init[n])
            speed.append(speed_init[n][0])
    Env_battery_update = Env_battery([[0, 0]], [[0, 0]], [[0, 0]],[],[],[[0, 0]])
    RL = DeepQNetwork(Env_battery_update.n_actions, Env_battery_update.n_states, learning_rate=0.00025, reward_decay=0.99, e_greedy=1, replace_target_iter=200, memory_size=1000)
    
    drv_cyle_length = len(data_pd) #for aging parameter update at end of each cycle 
    aging_flag = 0   
    
    for episode in range(80):
        HP_power_vector = [[0, 0]]
        HE_power_vector = [[0, final_data[0][1]]]
        HEcur = []
        HPcur = []
        cost_total = 0
        reward_total = 0
        #observation = Env_battery([[0, 0]]).reset()
        for time in range(len(final_data)):
            if time == 0:
                print(time)
                observation = Env_battery([[0, 0]], [[0, 0]], [[0, 0]],[],[],[[0, 0]]).reset()
                HE_soc = []
                HP_soc = []
                HE_Cap=12240            #HE battery initialization
                HE_R0=0.0248
                HE_R1=0.0081
                HE_R2=0.0282
                HE_C1=224.4569
                HE_C2=1929.2
                HE_Temp_cell=300
                HE_SOC=0.9
                HE_V1=0
                HE_V2=0
                HE_OCV=4.1
                HE_Vt=4.1
                HE_Xk=Xk = [[0.9], [0], [0]]
                HP_Cap=10440            #HP battery initialization
                HP_R0=4.5588e-3
                HP_R1=0.0013
                HP_R2=0.0040    
                HP_C1=2249.5
                HP_C2=18869
                HP_Temp_cell=300
                HP_SOC=0.5
                HP_V1=0
                HP_V2=0
                HP_OCV=4.1
                HP_Vt=4.1
                HP_Xk=Xk = [[0.5], [0], [0]]
            else:
                print(time)
                Env_battery_update = Env_battery(final_data[0:time], HE_power_vector , HP_power_vector, HEcur, HPcur, speed[0:time])
                #RL = DeepQNetwork(Env_battery_update.n_actions, Env_battery_update.n_states, learning_rate=0.01, reward_decay=0.9, e_greedy=0.9, replace_target_iter=200, memory_size=2000)
                action = RL.choose_action(observation)
                
                # RL take action and get next observation and reward
                observation_, reward, HE_power_vector, HP_power_vector, HEcur, HPcur\
,HE_Cap, HE_R0, HE_R1, HE_R2, HE_C1, HE_C2, HE_Temp_cell, HE_SOC, HE_V1, HE_V2, HE_OCV, HE_Vt, HE_Xk\
,HP_Cap, HP_R0, HP_R1, HP_R2, HP_C1, HP_C2, HP_Temp_cell, HP_SOC, HP_V1, HP_V2, HP_OCV, HP_Vt, HP_Xk\
= Env_battery_update.step(action, time-1, HE_Cap, HE_R0, HE_R1, HE_R2, HE_C1, HE_C2, HE_Temp_cell, HE_SOC, HE_V1, HE_V2, HE_OCV, HE_Vt, HE_Xk\
,HP_Cap, HP_R0, HP_R1, HP_R2, HP_C1, HP_C2, HP_Temp_cell, HP_SOC, HP_V1, HP_V2, HP_OCV, HP_Vt, HP_Xk\
,aging_flag, int(time/drv_cyle_length)*drv_cyle_length)  #####time stamp of action
                print("reward", reward)
                reward_total = reward_total + reward
                
                RL.store_transition(observation, action, reward, observation_)
                
                cost = RL.learn()
                cost_total = cost_total + cost
                # swap observation
                observation = observation_
                HE_soc.append(observation[0]) 
                HP_soc.append(observation[1])
                if isinstance(time/drv_cyle_length, int) and  time/drv_cyle_length > 1:
                  aging_flag = 1  
                else:
                  aging_flag = 0 
                    
                # break while loop when end of this episode
    # Env_battery.mainloop()
        #RL.plot_cost() 
        cost_final.append(cost_total)
        reward_final.append(reward_total)
		
    #plt.plot(np.arange(len(cost_final)),cost_final)
    #plt.ylabel('Cost')
    #plt.xlabel('Epoch ')
    #plt.show()
    #plt.savefig('/content/gdrive/My Drive/Colab Notebooks/cost_total.png')
    #plt.clf()
    
    #plt.plot(np.arange(len(reward_final)),reward_final)
    #plt.ylabel('Reward')
    #plt.xlabel('Epoch ')
    #plt.show()
    #plt.savefig('/content/gdrive/My Drive/Colab Notebooks/reward_total.png')
    #plt.clf()
    
    #plt.plot(np.arange(len(HE_soc)),HE_soc)
    #plt.plot(np.arange(len(HE_soc)),HP_soc)
    #plt.ylabel('SOC')
    #plt.xlabel('time')
    #plt.legend(['HE_SOC', 'HP_SOC'], loc='upper left')
    #plt.show()
    #plt.savefig('/content/gdrive/My Drive/Colab Notebooks/SOC.png')
    #plt.clf()
    #RL.save()


In [0]:
    plt.plot(np.arange(len(cost_final)),cost_final)
    plt.ylabel('Cost')
    plt.xlabel('Epoch ')
    plt.show()

In [0]:
    plt.plot(np.arange(len(reward_final)),reward_final)
    plt.ylabel('Reward')
    plt.xlabel('Epoch ')
    plt.show()

In [0]:
    plt.plot(np.arange(len(HE_soc)),HE_soc)
    plt.plot(np.arange(len(HE_soc)),HP_soc)
    plt.ylabel('SOC')
    plt.xlabel('time')
    plt.legend(['HE_SOC', 'HP_SOC'], loc='upper left')
    plt.show()